# Table Question Answering with Qdrant

In this notebook we will see how we can use Qdrant to make a Table Question Answering(Table QA) system i.e., a user's question will be answered from a table which contain the answer to that question. Qdrant is a vector Database powering the next generation of AI applications with advanced and high-performant vector similarity search technology.

The major steps involved in building Table QA system are:

- Loading a dataset which contains tables with information.
- Making vector embeddings of the tables.
- Storing the embedded tables in Qdrant collection.
- Initializing a table reader model which will read the output of qdrant after querying.
- System is ready we can now query the system.

## Install Dependencies

In [1]:
!pip install -qU datasets==2.13.1 qdrant-client==1.3.1 sentence-transformers==2.2.2 torch-scatter==2.1.1 openai==0.27.8

## Import libraries

In [2]:
from datasets import load_dataset
import pandas as pd
import torch
import openai
import os
from qdrant_client import QdrantClient
from pathlib import Path
from time import sleep
from qdrant_client.http import models
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load the Dataset

We will use the `wikitablequestions` dataset from Huggingface datasets. The WikiTableQuestions dataset is a large-scale dataset for the task of question answering on semi-structured tables. It contains about 11k tables.

In [113]:
# load the dataset from huggingface datasets hub
data = load_dataset("wikitablequestions", "random-split-1", split="train")
data

Found cached dataset wikitablequestions (C:/Users/karti/.cache/huggingface/datasets/wikitablequestions/random-split-1/1.0.2/4845d83f334ed5e4a8e0420731e64d57f696feb62e7d3a47b84037864fb8317c)


Dataset({
    features: ['id', 'question', 'answers', 'table'],
    num_rows: 11321
})

In [4]:
data[3]

{'id': 'nt-5',
 'question': 'how many people stayed at least 3 years in office?',
 'answers': ['4'],
 'table': {'header': ['',
   'Name',
   'Took office',
   'Left office',
   'Party',
   'Notes/Events'],
  'rows': [['11',
    'William McCreery',
    'March 4, 1803',
    'March 3, 1809',
    'Democratic Republican',
    ''],
   ['12',
    'Alexander McKim',
    'March 4, 1809',
    'March 3, 1815',
    'Democratic Republican',
    ''],
   ['13',
    'William Pinkney',
    'March 4, 1815',
    'April 18, 1816',
    'Democratic Republican',
    'Resigned to accept position as Minister Plenipotentiary to Russia'],
   ['14',
    'Peter Little',
    'September 2, 1816',
    'March 3, 1823',
    'Democratic Republican',
    ''],
   ['14',
    'Peter Little',
    'March 4, 1823',
    'March 3, 1825',
    'Jacksonian DR',
    ''],
   ['14', 'Peter Little', 'March 4, 1825', 'March 3, 1829', 'Adams', ''],
   ['15',
    'Benjamin C. Howard',
    'March 4, 1829',
    'March 3, 1833',
    'Jackson

We only require the table section from the dataset. We will extract the table section into a list of pandas dataframe.

In [5]:
# store all tables in the tables list
tables = []
# loop through the dataset and convert tabular data to pandas dataframes
for table_data in data:
    header = table_data["table"]["header"]
    rows = table_data["table"]["rows"]
    table_df = pd.DataFrame(rows, columns=header)
    tables.append(table_df)

In [6]:
tables[7]

,Year,Competition,Venue,Position,Notes
0,1996,Olympic Games,"Atlanta, United States",36th (q),5.55 m
1,1998,Asian Games,"Bangkok, Thailand",8th,6.07 m
2,1999,World Championships,"Seville, Spain",23rd (q),6.40 m
3,2000,Olympic Games,"Sydney, Australia",14th (q),6.57 m
4,2001,World Championships,"Edmonton, Canada",13th (q),6.46 m
5,2002,Asian Championships,"Colombo, Sri Lanka",1st,6.61 m
6,2002,Asian Games,"Busan, South Korea",3rd,6.30 m
7,2003,World Championships,"Paris, France",23rd (q),6.13 m
8,2003,Asian Championships,"Manila, Philippines",6th,6.23 m
9,2004,Olympic Games,"Athens, Greece",11th,6.53 m


Also we will trim all tables to 5 rows.

In [7]:
for i, table in enumerate(tables):
    tables[i] = table.head(5)
tables[0]

,Year,Division,League,Regular Season,Playoffs,Open Cup,Avg. Attendance
0,2001,2,USL A-League,"4th, Western",Quarterfinals,Did not qualify,"7,169"
1,2002,2,USL A-League,"2nd, Pacific",1st Round,Did not qualify,"6,260"
2,2003,2,USL A-League,"3rd, Pacific",Did not qualify,Did not qualify,"5,871"
3,2004,2,USL A-League,"1st, Western",Quarterfinals,4th Round,"5,628"
4,2005,2,USL First Division,5th,Quarterfinals,4th Round,"6,028"


## Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things: 
- Generate embeddings for all tables (table vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)
The retriever will generate embeddings in a way that the questions and tables containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and tables to find the tables that contain potential answers to our question.

We will use OpenAi model `text-embedding-ada-002` to create embeddings. First, Initialize connection to OpenAi:

In [9]:
# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"
openai.Engine  # check we have authenticated

openai.api_resources.engine.Engine

In [21]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch",
    ],
    engine=embed_model,
)
len(res["data"][0]["embedding"]), len(
    res["data"][1]["embedding"]
)  # the model produce embeddings of dimension 1536

(1536, 1536)

The openai model creates vector embeddings of 1536 dimensions.

We will need to convert the tables dataframes into list of csv to create embeddings.

In [23]:
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed

In [24]:
# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)
# display the formatted table
processed_tables[8]

'No.,Temple,Honzon (main image),City/Town/Village,Prefecture\r\n1,Ryōzen-ji (霊山寺),Shaka Nyorai,Naruto,Tokushima Prefecture\r\n2,Gokuraku-ji (極楽寺),Amida Nyorai,Naruto,Tokushima Prefecture\r\n3,Konsen-ji (金泉寺),Shaka Nyorai,Itano,Tokushima Prefecture\r\n4,Dainichi-ji (大日寺),Dainichi Nyorai,Itano,Tokushima Prefecture\r\n5,Jizō-ji (地蔵寺),Enmei Jizō Bosatsu,Itano,Tokushima Prefecture\r\n'

The processed tables are not very readable for humans but the embedding model can understand and create embeddings while capturing the semantic meaning of information in the tables.

## Initialize Qdrant client

In [25]:
# Initialize Qdrant client

current_folder = Path.cwd()  # Get the current folder
qdrant_folder = current_folder / "qdrant"
qdrant_folder.mkdir()  # Create qdrant folder to store collection

client = QdrantClient(path=qdrant_folder.resolve())  # path to new qdrant folder

## Create collection
We need to create a new collection in our Qdrant vector database which will store all the vector embeddings. While creating collection we will also specify the dimension of vectors collection is supposed to store and the metric which will be used to calculate similarity between query vector and table vector.

In [26]:
# you can choose any name for the collection
collection_name = "table-qa"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in collections:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=1536,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='table-qa')]


## Generate Embeddings -> Store in Qdrant
When passing the documents to Qdrant, we need an:

- id (a unique integer value),
- table embedding, and
- payload for each document representing table passages in the dataset. The payload is a dictionary containing data relevant to our embeddings.

This time we will not use payload as we will directly use id from Qdrant results to get the tables from our list of tables dataframe.

In [34]:
%%time

batch_size = 1024  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for i in tqdm(range(0, len(processed_tables), batch_size)):
    i_end = min(i + batch_size, len(processed_tables))  # find end of batch
    batch = processed_tables[i:i_end]  # extract batch
    ids = list(range(i, i_end))  # create unique IDs
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=batch, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=batch, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record["embedding"] for record in res["data"]]

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=embeds),
    )

collection_vector_count = client.get_collection(
    collection_name=collection_name
).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(processed_tables)

  0%|          | 0/12 [00:00<?, ?it/s]

Vector count in collection: 11321
CPU times: total: 1min 2s
Wall time: 3min 5s


Now we can search  with query vectors.

In [47]:
query = "name a city with more than 12,000,000 people."
# generate embedding for the query
res = openai.Embedding.create(input=[query], engine=embed_model)
encoded_query = res["data"][0]["embedding"]

res = client.search(  # search qdrant collection
    collection_name=collection_name,
    query_vector=encoded_query,
    limit=1,
)
res

[ScoredPoint(id=11222, version=0, score=0.827129729226599, payload={}, vector=None)]

After searching qdrant collection, we get the similarity score of the results. We also get the **id** which we can use to retrive the table from **tables** list.

In [46]:
id = res[0].id
tables[id].head()

,Rank,City,Population (2011),Population (2001),State/Territory
0,1,Mumbai,"12,478,447","11,978,450",Maharashtra
1,2,Delhi,"11,007,835","9,879,172",Delhi
2,3,Bangalore,"8,425,970","5,438,065",Karnataka
3,4,Hyderabad,"6,809,970","3,637,483",Andhra Pradesh
4,5,Ahmedabad,"5,570,585","3,520,085",Gujarat


The result we get from qdrant will contain the answer to our query but we need to fetch the answer from the table using a table reader model.

## Initialize Table Reader
We will use `neulab/omnitab-large-finetuned-wtq` as our table reader model. OmniTab is a table-based QA model which is fine-tuned on WikiTableQuestions.

In [50]:
tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large-finetuned-wtq")
model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large-finetuned-wtq")

Let's write a function `get_answer_from_table` which will return the answer given a table and a query.

In [114]:
def get_answer_from_table(table, query):
    encoding = tokenizer(table=table, query=query, return_tensors="pt")
    outputs = model.generate(**encoding, max_new_tokens=100)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [52]:
query = "name a city with more than 12,000,000 people."
get_answer_from_table(tables[id].head(), query)

[' Mumbai']

## Querying

We also need a function to search in qdrant and return a table in the format as expected by `get_answer_from_table` function. We name this function `query_qdrant`.

In [62]:
def query_qdrant(query):
    # generate embedding for the query
    res = openai.Embedding.create(input=[query], engine=embed_model)
    encoded_query = res["data"][0]["embedding"]
    result = client.search(
        collection_name=collection_name,
        query_vector=encoded_query,
        limit=1,
    )  # search qdrant collection for context passage with the answer
    # return the relevant table from the tables list
    id = result[0].id
    return tables[id].head()

In [121]:
query = "which person has the longest tenure as president? and how much?"
table = query_qdrant(query)
table

,order in\nposition,president of\nthe republic,length of\nthe presidential\naction,rank by\nlength\nof term(s),notes
0,1,K. J. Ståhlberg,"5 years, 220 days",8,1 term serving
1,2,Lauri Kristian Relander,"6 years, 0 days\none leap year (1928)",6,1 term serving
2,3,P. E. Svinhufvud,"5 years, 364 days\ntwo leap years (1932 and 1936)",6,1 term serving
3,4,Kyösti Kallio,"3 years, 293 days",9,* resigned due to poor health
4,5,Risto Ryti,"3 years, 229 days",10,* resigned due to agreement with Nazi Germany\...


In [116]:
get_answer_from_table(table, query)

[' Lauri Kristian Relander, 6 years, 0 days']

In [117]:
query = "How many countries have gdp above 100 billion US $ ?"
table = query_qdrant(query)
table.head()

,Country,Total GDP (nominal)\n(billion US$),"GDP per capita\n(US$, PPP)","GDP Growth,\n2007-2011\n(in %)",HDI
0,Algeria,188.7,"8,715",2.7,0.733
1,Angola,101.0,"5,930",9.1,0.446
2,Benin,7.3,"1,628",3.9,0.437
3,Botswana,17.6,"14,753",3.0,0.654
4,Burkina Faso,10.2,"1,310",4.9,0.370


In [118]:
get_answer_from_table(table, query)

[' 2']

In [119]:
query = "What movie made the most money?"
table = query_qdrant(query)
table.head()

,Rank,Title,Studio,Actors,Gross
0,1.,Star Wars*,Lucasfilm/20th Century Fox,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet...","$460,998,007"
1,2.,Smokey and the Bandit,Universal/Rastar,"Burt Reynolds, Sally Field, Jackie Gleason, Je...","$300,000,000"
2,3.,Close Encounters of the Third Kind*,Columbia,"Richard Dreyfuss, Teri Garr, Melinda Dillon an...","$166,000,000"
3,4.,Saturday Night Fever,Paramount,John Travolta and Karen Lynn Gorney,"$139,486,124"
4,5.,The Goodbye Girl,MGM/Warner Bros./Rastar,"Richard Dreyfuss, Marsha Mason and Quinn Cummings","$102,000,000"


In [120]:
get_answer_from_table(table, query)

[' Star Wars']

The result looks pretty good, you can query it on more questions.

In [ ]:
client.delete_collection(collection_name=collection_name)